### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
import trajoptpy.math_utils as mu
import multiprocessing as mp
from multiprocessing import Lock
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/dual_arm_random_init/'

### Setting the trajopt

In [2]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("robots/pr2-beta-static.zae")
env.Load("../env/bookshelves.env.xml")
env.SetDefaultViewer()

True

### Setting the robot

In [3]:
def mirror_arm_joints(x):
    "mirror image of joints (r->l or l->r)"
    return [-x[0],x[1],-x[2],x[3],-x[4],x[5],-x[6]]

In [4]:
robot = env.GetRobots()[0]
left_arm = robot.GetManipulator("leftarm")
right_arm = robot.GetManipulator("rightarm")

robot.SetActiveDOFs(np.r_[right_arm.GetArmIndices(), left_arm.GetArmIndices()])
coeffs = [1,1,1,1,1,1,1,  1,1,1,1,1,1,1,] #coefficients for joint velocity cost

dof = len(robot.GetActiveDOFValues())
left_dof_away = np.array([ 0.5646,  1.1371, -0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
right_dof_away = np.array([ -0.5646,  1.1371, 0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
robot.SetDOFValues(left_dof_away, left_arm.GetArmIndices())
robot.SetDOFValues(right_dof_away, right_arm.GetArmIndices())
init_joint0 = robot.GetActiveDOFValues()

In [5]:
left_waypoints = [
    [-0.043379, 0.103374, -1.6, -2.27679, 3.02165, -2.03223, -1.6209], #chest fwd
    [-0.18199, -0.088593, -1.6, -2.08996, 3.04403, -0.41007, -1.39646],# side fwd
    [-0.0428341, -0.489164, -0.6, -1.40856, 2.32152, -0.669566, -2.13699],# face up
    [0.0397607, 1.18538, -0.8, -0.756239, -2.84594, -1.06418, -2.42207]# floor down
]

right_waypoints = []
for w in left_waypoints:
    right_waypoints.append(mirror_arm_joints(w))
    
waypoints = [init_joint0.copy()]
for i in range(len(left_waypoints)):
    waypoints.append(right_waypoints[i]+left_waypoints[i])

### Set the IK Solver 

In [6]:
hmat_target = np.array([[ 0.9689,  0.    , -0.2474,  0.4918],
       [-0.    ,  1.    ,  0.    ,  0.912 ],
       [ 0.2474, -0.    ,  0.9689,  0.8947],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])
quat_target = openravepy.quatFromRotationMatrix(hmat_target).tolist()
xyz_target = hmat_target[0:3,3].tolist()

# BEGIN IK for all manips
manips = [left_arm,right_arm]
for manip in manips:
    robot.SetActiveManipulator(manip)
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(
        robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        ikmodel.autogenerate()   
    
target_joint = ku.ik_for_link(hmat_target, left_arm, "l_gripper_tool_frame",
    filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
# END ik

### Build / Retrieve Dataset

In [7]:
def generate_inputs(n = 100, filename = 'data.pkl'):
    x = []
    target_joints = []
    for i in range(n):
        _,_,_,_,init_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(init_joint)
        _,_,_,_,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        x.append(np.concatenate([init_joint, target_joint]))
        
        if i%100 == 0:
            print i

    x = np.vstack(x)
    data = dict()
    data['x'] = x
    input_file = open(filename, 'wb')
    pickle.dump(data, input_file)
    input_file.close()
    return data

In [8]:
def build_database(x_init, y_init, num_traj, limits,  predictor=None, n_steps = 30, x_train = None):
    num_plan = 0
    comp_times = []
    costs = []
    tic = time.time()
    
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        if x_train is None:
            #Get a random pose as init
            _,_,_,_,init_joint = get_random_pose_both(env,limits,right_arm,left_arm)
            robot.SetActiveDOFValues(init_joint)
            #Get a random pose as target
            _,_,_,_,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
            robot.SetActiveDOFValues(target_joint)
        else:
            x_cur = x_train[num_plan]
            init_joint = x_cur[:dof]
            target_joint = x_cur[dof:]
            
        #plan
        for func in predictor:
            x_cur = np.atleast_2d(np.concatenate([init_joint,target_joint]))
            if isinstance(func, Straight_Regressor):
                traj,_ = func.predict(init_joint, target_joint)
            else:
                traj,_ = func.predict(x_cur)
            if func.is_transform == 'PCA':
                traj = func.pca.inverse_transform(traj)
            traj = traj.reshape(-1,dof)
            traj[0,:] = init_joint
            robot.SetActiveDOFValues(init_joint)

            request_traj = define_request(time_step =n_steps,coeffs = coeffs,init_type='given_traj',constraint_type='joint')
            request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
            request_traj = set_init(request_traj, 'given_traj', traj)
            duration, result = run_opt(request_traj, env)    

            #check traj result
            traj = result.GetTraj()
            if check_traj(env,result, target_joint):
                print 'Planning is successfull!'
                x_init.append(np.concatenate([init_joint, target_joint]))
                y_init.append(traj.flatten())
                comp_times.append(duration)
                costs.append(result.GetCosts()[0][1])
                break
            else:
                print('Fail to find good solution!')
                continue

        num_plan += 1
        
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

### Generate the possible inputs

In [9]:
to_create_input = False
if to_create_input:
    data_train = generate_inputs(n = 5000, filename = FILENAME + 'data_train.pkl')
else:
    data_train = pickle.load(open(FILENAME + 'data_train.pkl', 'rb'))
    
x_train = data_train['x']

In [53]:
to_build_database = True
to_augment_database = True
n_steps = 30
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

num_traj = 500
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        x_train = x_train[2*len(x_init):]
        num_traj += len(x_init)  
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, x_train = x_train, predictor = [straight])
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)

In [54]:
print x_init.shape

(500, 28)


### Reduce the data sample

In [42]:
N_REDUCED = 350
FILENAME_REDUCED = FILENAME + str(N_REDUCED) + '_'
x_init = x_init[0:N_REDUCED]
y_init = y_init[0:N_REDUCED]

### Apply Dimensionality Reduction to the data

In [43]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Check Database 

### Build Function Approximator

In [44]:
dim_input = len(x_init[0])
is_load_regressor = False

nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
gpy_pca.pca = y_pca
bgmr = DP_GLM_Regressor()
bgmr_pca = DP_GLM_Regressor(is_transform='PCA')
bgmr_pca.pca = y_pca

if is_load_regressor:
    nn.load_from_file(FILENAME_REDUCED + 'nn.pkl')
    gpy.load_from_file(FILENAME_REDUCED + 'gpy.pkl')
    gpy_pca.load_from_file(FILENAME_REDUCED + 'gpy_pca.pkl')
    bgmr.load_from_file(FILENAME_REDUCED + 'bgmr.pkl')
    bgmr_pca.load_from_file(FILENAME_REDUCED + 'bgmr_pca.pkl')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME_REDUCED + 'nn.pkl')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init)
    gpy.save_to_file(FILENAME_REDUCED + 'gpy.pkl')

    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME_REDUCED + 'gpy_pca.pkl')

    print 'Planning for bgmr'
    bgmr.fit(x_init,y_init)
    bgmr.save_to_file(FILENAME_REDUCED + 'bgmr.pkl')
    
    print 'Planning for bgmr PCA'
    bgmr_pca.fit(x_init,y_init_reduced)
    bgmr_pca.save_to_file(FILENAME_REDUCED + 'bgmr_pca.pkl')

Planning for NN
Planning for GPY
Optimization restart 1/10, f = 258643.867422
Optimization restart 2/10, f = -74971.2419296
Optimization restart 3/10, f = 258643.710016
Optimization restart 4/10, f = -74818.798576
Optimization restart 5/10, f = -74971.30049
Optimization restart 6/10, f = -74970.0134363
Optimization restart 7/10, f = -74971.3845431
Optimization restart 8/10, f = -74971.1358389
Optimization restart 9/10, f = -74971.173124
Optimization restart 10/10, f = -74971.3851514
Planning for GPY PCA
Optimization restart 1/10, f = 42541.9804099
Optimization restart 2/10, f = 9496.58840611
Optimization restart 3/10, f = 9496.58845447
Optimization restart 4/10, f = 9496.60240877
Optimization restart 5/10, f = 16463.5110695
Optimization restart 6/10, f = 17183.4244478
Optimization restart 7/10, f = 9609.55625925
Optimization restart 8/10, f = 9496.5708459
Optimization restart 9/10, f = 9496.57777232
Optimization restart 10/10, f = 9496.63052726
Planning for bgmr
Planning for bgmr PCA


### Creating the test cases

In [45]:
create_test = False

if create_test:
    num_test = 300
    data_test = generate_inputs(num_test, FILENAME + 'data_test.pkl')
    x_test = data_test['x']
else:
    #load data
    test_file = open(FILENAME + 'data_test.pkl', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x']
    num_test = len(x_test)

### Setting up Parallel Programming for Ensemble Method

In [46]:
from joblib import Parallel, delayed
mutex = Lock()
   
def run_opt_special(request):
    robot.SetActiveDOFValues(init_joint)
    s = json.dumps(request) 
    prob = trajoptpy.ConstructProblem(s, env)
    tic = time.time()
    try:
        result = trajoptpy.OptimizeProblem(prob) # do optimization
    except:
        print 'Fault in optimization'
        return False, None, None, None
    toc = time.time()
    print("Optimization is completed in {} s!".format(toc-tic))
    duration = toc-tic
    
    #check the result
    try:
        success = check_traj(env,result, target_joint)    
        result_new = dict()
        result_new['traj'] = result.GetTraj()
        result_new['costs'] = result.GetCosts()[0][1]
        cost = result.GetCosts()[0][1]
        result_new['status'] = result.GetStatus()
        return success, duration, cost, result_new
    except:
        'Got an exception when checking the trajectory'
        return False, None, None, None
    
def function_callback(result):
    with mutex:
        print 'Obtain result:'
        print result[0]
        global count_proc
        global num_proc
        count_proc+= 1
        global ensemble_result
        ensemble_result = result
        print count_proc, num_proc
        if result[0]:
            print 'Terminating the mp'
            pool.terminate()
            return
        if count_proc == num_proc:
            print 'All fails, terminating the mp'
            pool.terminate()

### Evaluation

In [47]:
method_names = ['STD    ', 'NN      ', 'GPR    ', 'GPR_PCA', 'BGMR', 'BGMR_PCA', 'ensemble', 'waypoints']
methods = [straight, nn, gpy, gpy_pca, bgmr, bgmr_pca]
#method_names = ['STD    ', 'NN      ', 'GPR    ',  'ensemble', 'waypoints']
#methods = [straight, nn, gpy]

results = dict()
ensemble_result = None
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []

result,success, duration = dict(),dict(),dict()
num_test = 100
for it in range(0,num_test):
    print('{}th Planning'.format(it))
    
    #setting up the problem case
    index = np.random.randint(0,len(x_test))
    index = it
    x_cur = x_test[index:index+1,:].flatten()
    init_joint = x_cur.copy()[0:dof]
    target_joint = x_cur.copy()[dof:]
    robot.SetActiveDOFValues(init_joint)
    
    
    x_cur = np.atleast_2d(np.concatenate([init_joint,target_joint]))
    request_trajs = []
    for i,method in enumerate(methods):
        request_traj = define_request(coeffs = coeffs,init_type='given_traj',constraint_type='joint')
        request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,_ = method.predict(init_joint, target_joint)
        else:
            traj,cov = method.predict(x_cur)
        if method.is_transform == 'PCA':
            traj = method.pca.inverse_transform(traj)
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint
        robot.SetActiveDOFValues(init_joint)
        request_traj = set_init(request_traj, 'given_traj', traj)
        success[method_name], duration[method_name], _, result[method_name] = run_opt_special(request_traj)    
        request_trajs.append(request_traj)

    
    #ensemble methods   
    print("Start ensemble")
    num_proc = len(request_trajs)
    pool = mp.Pool(processes = num_proc)
    count_proc = 0
    ticc = time.time()
    for request in request_trajs:
        pool.apply_async(run_opt_special, args = (request, ), callback = function_callback)
    time.sleep(5)
    pool.terminate()
    pool.close()
    pool.join()
    tocc = time.time()
    print 'Ensemble takes ' + str(tocc-ticc)
    results['ensemble']['costs'].append(ensemble_result[2])
    results['ensemble']['successes'].append(ensemble_result[0])      
    results['ensemble']['comp_times'].append(tocc-ticc)
    
    #multiple_waypoints
    print("Start multiple waypoints")
    cur_waypoints = list(waypoints)
    cur_waypoints.append(0.5*(init_joint+target_joint))
    num_proc = len(cur_waypoints)
    pool = mp.Pool(processes = num_proc)
    count_proc = 0
    ticc = time.time()
    #create request_trajs
    request_trajs = []
    for j in range(num_proc):
        waypoint = cur_waypoints[j]
        request_traj = define_request(coeffs = coeffs,init_type='given_traj',constraint_type='joint')
        request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
        method_name = 'waypoints'
        traj,_ = straight.predict_with_waypoint(init_joint, target_joint, waypoint, n_steps/2 )
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint
        robot.SetActiveDOFValues(init_joint)
        request_traj = set_init(request_traj, 'given_traj', traj)
        request_trajs.append(request_traj)
    
    #run request_trajs
    for request in request_trajs:
        pool.apply_async(run_opt_special, args = (request, ), callback = function_callback)
    time.sleep(5)
    pool.terminate()
    pool.close()
    pool.join()
    tocc = time.time()
    print 'Multiple waypoints takes ' + str(tocc-ticc)
    results['waypoints']['costs'].append(ensemble_result[2])
    results['waypoints']['successes'].append(ensemble_result[0])      
    results['waypoints']['comp_times'].append(tocc-ticc)
    
    #Record the result
    for method_name in method_names[:-2]:
        results[method_name]['costs'].append(result[method_name]['costs'])#.GetCosts())#[0][1])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])

0th Planning
Optimization is completed in 3.39662408829 s!
There is a collision within the trajectory!
Optimization is completed in 2.06759309769 s!
There is a collision within the trajectory!
Optimization is completed in 7.6225810051 s!
There is a collision within the trajectory!
Optimization is completed in 0.899078845978 s!
Optimization is completed in 3.07768297195 s!
Optimization is completed in 1.22614407539 s!
Start ensemble
Optimization is completed in 1.39654898643 s!
There is a collision within the trajectory!
Optimization is completed in 1.64043188095 s!
Obtain result:
False
1 6
Obtain result:
True
2 6
Terminating the mp
Ensemble takes 1.94259619713
Start multiple waypoints
Optimization is completed in 1.49176216125 s!
Obtain result:
True
1 6
Terminating the mp
Multiple waypoints takes 1.74363493919
1th Planning
Optimization is completed in 1.44331789017 s!
Optimization is completed in 0.457035064697 s!
There is a collision within the trajectory!
Optimization is completed in

### Printing Results

In [49]:
from planning_util import *

In [50]:
print_result(results, method_names)

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD     	& 78.000 	& 0.97$\pm$0.67 	& 1.16$\pm$0.67 	 \\
NN       	& 66.000 	& 1.38$\pm$0.75 	& 1.63$\pm$1.06 	 \\
GPR     	& 87.000 	& 0.89$\pm$0.45 	& 1.25$\pm$0.78 	 \\
GPR_PCA 	& 84.000 	& 0.91$\pm$0.47 	& 1.25$\pm$0.76 	 \\
BGMR 	& 75.000 	& 1.06$\pm$0.54 	& 1.24$\pm$0.72 	 \\
BGMR_PCA 	& 77.000 	& 1.04$\pm$0.44 	& 1.24$\pm$0.67 	 \\
ensemble 	& 96.000 	& 1.17$\pm$0.42 	& 1.45$\pm$0.98 	 \\
waypoints 	& 93.000 	& 1.53$\pm$0.65 	& 1.75$\pm$1.42 	 \\


### Save Result

In [51]:
pickle.dump(results,open(FILENAME_REDUCED + 'result.pkl', 'wb') )

### Load Result 

In [ ]:
results = pickle.load(open(FILENAME_REDUCED + 'result.pkl', 'rb'))

### Printing Results Tied to Standard Success

In [21]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    
    print('{0}: \t& {1:.3f} \t& {2:.2f} \t& {3:.3f} \\\\'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success))#, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD    : 	& 1.000 	& 0.85 	& 1.309 \\
NN      : 	& 0.693 	& 1.17 	& 1.488 \\
GPR    : 	& 0.910 	& 0.80 	& 1.282 \\
GPR_PCA: 	& 0.937 	& 0.82 	& 1.291 \\
BGMR: 	& 0.899 	& 0.80 	& 1.307 \\
BGMR_PCA: 	& 0.862 	& 0.88 	& 1.329 \\
ensemble: 	& 1.000 	& 0.99 	& 1.345 \\
waypoints: 	& 0.979 	& 1.29 	& 1.595 \\
